In [2]:
!pip install open3d Trimesh numpy-stl

In [3]:
from stl import mesh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np
import random
import plotly.graph_objects as go

In [4]:
!pip install pyvista

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 MB 4.4 MB/s eta 0:00:00


# Reading data from tumored brain

In [5]:
# Load the mesh
mesh0 = mesh.Mesh.from_file('Data/000000_tumoredbrain.stl')

# Convert the mesh to a point cloud by sampling points from the mesh surface
points = mesh0.points[:,:3]
idx=random.sample(range(len(points)), 10000)
point_cloud_0=np.array(points[idx])

fig = go.Figure(data=[go.Scatter3d(
    x=point_cloud_0[:, 0],
    y=point_cloud_0[:, 1],
    z=point_cloud_0[:, 2],
    mode='markers',
    marker=dict(size=2, color=point_cloud_0[:, 0], colorscale='Viridis', opacity=0.8)
)])

fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'),
    title="3D Point Cloud Visualization"
)

fig.show()


In [6]:
mesh1= mesh.Mesh.from_file('Data/000001_tumoredbrain.stl')

points = mesh1.points[:,:3]
idx=random.sample(range(len(points)), 10000)
point_cloud_1=np.array(points[idx])

fig = go.Figure(data=[go.Scatter3d(
    x=point_cloud_1[:, 0],
    y=point_cloud_1[:, 1],
    z=point_cloud_1[:, 2],
    mode='markers',
    marker=dict(size=2, color=point_cloud_1[:, 0], colorscale='Viridis', opacity=0.8)
)])

fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'),
    title="3D Point Cloud Visualization"
)

fig.show()


In [7]:
# Load the mesh
mesh5 = mesh.Mesh.from_file('Data/000005_tumoredbrain.stl')

# Convert the mesh to a point cloud by sampling points from the mesh surface
points = mesh5.points[:,:3]
idx=random.sample(range(len(points)), 10000)
point_cloud_5=np.array(points[idx])

fig = go.Figure(data=[go.Scatter3d(
    x=point_cloud_5[:, 0],
    y=point_cloud_5[:, 1],
    z=point_cloud_5[:, 2],
    mode='markers',
    marker=dict(size=2, color=point_cloud_5[:, 0], colorscale='Viridis', opacity=0.8)
)])

fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'),
    title="3D Point Cloud Visualization"
)

fig.show()


# Landmark selection by k-means

In [8]:
# TESTING SEGMENTATION AND CHOSING LANDMARK BY KMEANS
from sklearn.cluster import KMeans

n_landmarks = 3000  # number of landmarks same as in paper
kmeans1 = KMeans(n_clusters=n_landmarks)
kmeans1.fit(point_cloud_1)
landmarks_1 = kmeans1.cluster_centers_

kmeans0 = KMeans(n_clusters=n_landmarks)
kmeans0.fit(point_cloud_0)
landmarks_0 = kmeans0.cluster_centers_

kmeans5 = KMeans(n_clusters=n_landmarks)
kmeans5.fit(point_cloud_5)
landmarks_5 = kmeans5.cluster_centers_



In [9]:
fig = go.Figure(data=[go.Scatter3d(
    x=landmarks_1[:, 0],
    y=landmarks_1[:, 1],
    z=landmarks_1[:, 2],
    mode='markers',
    marker=dict(size=2, color=landmarks_1[:, 0], colorscale='Viridis', opacity=0.8)
)])

fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'),
    title="3D Point Cloud Visualization"
)

fig.show()


In [11]:
# Plot landmarks
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=landmarks_1[:, 0],
    y=landmarks_1[:, 1],
    z=landmarks_1[:, 2],
    mode='markers',
    marker=dict(size=3, color='red', opacity=0.8),
    name="Fig 5"
))

fig.add_trace(go.Scatter3d(
    x=landmarks_0[:, 0],
    y=landmarks_0[:, 1],
    z=landmarks_0[:, 2],
    mode='markers',
    marker=dict(size=3, color='blue', opacity=0.8),
    name="Fig 0"
))

fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'),
    title="3D Point Cloud Visualization"
)

fig.show()

In [12]:
# Plot landmarks
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=landmarks_5[:, 0],
    y=landmarks_5[:, 1],
    z=landmarks_5[:, 2],
    mode='markers',
    marker=dict(size=3, color='green', opacity=0.8),
    name="Fig 1"
))


fig.add_trace(go.Scatter3d(
    x=landmarks_0[:, 0],
    y=landmarks_0[:, 1],
    z=landmarks_0[:, 2],
    mode='markers',
    marker=dict(size=3, color='blue', opacity=0.8),
    name="Fig 0"
))


fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'),
    title="3D Point Cloud Visualization"
)

fig.show()

# Rigid registration

In [ ]:
def compute_nearest_neighbor(X, Y):
    # Compute the distance from each element of X to each element of Y
    diff = ((X[:,None,:]-Y)**2).sum(axis=2)

    # get the indexes of the min
    argmin = np.argmin(diff, axis=1)

    return argmin


def compute_rigid_transform(X_source, X_target):
    # Compute translation
    t = X_target.mean(axis=0) - X_source.mean(axis=0)

    # Compute the matrix A
    A = (-t+X_target).T @ X_source

    # Solve procrustes problem to find rotation
    U, _, Vh = np.linalg.svd(A)
    R = U @ Vh

    # Fix the det signal
    if np.linalg.det(R) < 0:
        R[:, -1] *= -1

    return R, t

def transform_pointcloud(X,R,t):

    return X @ R.T + t

def icp_align(X_source, Y_target, n_iter=10):
    i=0
    while i < (n_iter):
        i+=1

        # First: find closest points from X on Y
        idx = compute_nearest_neighbor(X_source, Y_target)
        Y_target = Y_target[idx]

        # Find the best translation and rotation
        R, t = compute_rigid_transform(X_source, Y_target)

        # Align point clouds
        X_source = transform_pointcloud(X_source,R,t)

    X_aligned = X_source

    return X_aligned

aligned_landmarks_1 = icp_align(landmarks_1,landmarks_0,n_iter=100)
aligned_landmarks_5 = icp_align(landmarks_5,landmarks_0,n_iter=100)


In [ ]:
# Plot landmarks
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=aligned_landmarks_1[:, 0],
    y=aligned_landmarks_1[:, 1],
    z=aligned_landmarks_1[:, 2],
    mode='markers',
    marker=dict(size=3, color='red', opacity=0.8),
    name="Aligned 1"
))

# Add the second cluster
fig.add_trace(go.Scatter3d(
    x=landmarks_0[:, 0],
    y=landmarks_0[:, 1],
    z=landmarks_0[:, 2],
    mode='markers',
    marker=dict(size=3, color='blue', opacity=0.8),
    name="Cluster 0"
))

fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'),
    title="3D Point Cloud Visualization"
)

fig.show()

# Perform SSM/PCA

In [ ]:
from sklearn.decomposition import PCA
def perform_pca(clouds):
    # Stack all data and compute pca to extract most meaningful components
    flat_clouds = np.vstack(clouds)
    pca = PCA(n_components=3, svd_solver='full')
    pca.fit(flat_clouds)
    return pca


def visualize_clouds(clouds):
    colors = ['red', 'green', 'blue']
    plt.figure(figsize=(8, 8))
    ax = plt.axes(projection='3d')
    for i, cloud in enumerate(clouds):
        ax.scatter(cloud[:, 0], cloud[:, 1], cloud[:, 2], color=colors[i % len(colors)], label=f'Cloud {i+1}')
    ax.legend()
    plt.show()


In [ ]:
# Perform PCA on aligned shapes
aligned_clouds = [aligned_landmarks_1,aligned_landmarks_5,landmarks_0]
mean_shape = np.mean(aligned_clouds, axis=0)
variation = aligned_clouds - mean_shape
pca = perform_pca(variation)

# Get transformed shapes
# From main variation modes, obtain representation of the shapes
reconstructed_cloud = [mean_shape+pca.transform(cloud) for cloud in aligned_clouds]

reconstructed_cloud_1 = reconstructed_cloud[0]
reconstructed_cloud_5 = reconstructed_cloud[1]

In [ ]:
# Plot mean model
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=mean_shape[:, 0],
    y=mean_shape[:, 1],
    z=mean_shape[:, 2],
    mode='markers',
    marker=dict(size=3, color='pink', opacity=0.8),
    name="Mean cluster"
))

fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'),
    title="3D Point Cloud Visualization"
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=reconstructed_cloud_5[:, 0],
    y=reconstructed_cloud_5[:, 1],
    z=reconstructed_cloud_5[:, 2],
    mode='markers',
    marker=dict(size=3, color='green', opacity=0.8),
    name="Cluster 5"
))

fig.add_trace(go.Scatter3d(
    x=reconstructed_cloud_1[:, 0],
    y=reconstructed_cloud_1[:, 1],
    z=reconstructed_cloud_1[:, 2],
    mode='markers',
    marker=dict(size=3, color='red', opacity=0.8),
    name="Cluster 1"
))

fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'),
    title="3D Point Cloud Visualization"
)

fig.show()

# B-Spline registration

In [ ]:
!pip install SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 7.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
from scipy.ndimage import binary_fill_holes
import SimpleITK as sitk

# SimpleITK uses voxel images, so we need to voxelize them
def point_cloud_to_volume(point_cloud, grid_size=128):
    min_coords = np.min(point_cloud, axis=0)
    max_coords = np.max(point_cloud, axis=0)
    volume = np.zeros((grid_size, grid_size, grid_size), dtype=np.uint8)

    scale = (grid_size - 1) / (max_coords - min_coords)
    scaled_points = ((point_cloud - min_coords) * scale).astype(int)

    for x, y, z in scaled_points:
        volume[x, y, z] = 1

    return sitk.GetImageFromArray(volume)

voxel_grid_0 = point_cloud_to_volume(landmarks_0)
voxel_grid_1 = point_cloud_to_volume(aligned_landmarks_1)
voxel_grid_5 = point_cloud_to_volume(aligned_landmarks_5)
sitk.WriteImage(voxel_grid_0, "voxel_grid_0.nii")
sitk.WriteImage(voxel_grid_1, "voxel_grid_1.nii")
sitk.WriteImage(voxel_grid_5, "voxel_grid_5.nii")

In [ ]:
import SimpleITK as sitk
import sys
import os


def command_iteration(method, bspline_transform) :
    if method.GetOptimizerIteration() == 0:
        # The BSpline is resized before the first optimizer
        # iteration is completed per level. Print the transform object
        # to show the adapted BSpline transform.
        print(bspline_transform)


    print("{0:3} = {1:10.5f}".format(method.GetOptimizerIteration(),
                                     method.GetMetricValue()))



def command_multi_iteration(method) :
    # The sitkMultiResolutionIterationEvent occurs before the
    # resolution of the transform. This event is used here to print
    # the status of the optimizer from the previous registration level.
    if R.GetCurrentLevel() > 0:
        print("Optimizer stop condition: {0}".format(R.GetOptimizerStopConditionDescription()))
        print(" Iteration: {0}".format(R.GetOptimizerIteration()))
        print(" Metric value: {0}".format(R.GetMetricValue()))

    print("--------- Resolution Changing ---------")

fixed_image = sitk.ReadImage("voxel_grid_0.nii",sitk.sitkFloat32)
moving_image = sitk.ReadImage("voxel_grid_1.nii",sitk.sitkFloat32)

# B-spline transform
transform_domain_mesh_size=[8]*fixed_image.GetDimension()
initial_transform = sitk.BSplineTransformInitializer(fixed_image, transform_domain_mesh_size)

print("Initial Number of Parameters: {0}".format(initial_transform.GetNumberOfParameters()))


R = sitk.ImageRegistrationMethod()
R.SetMetricAsJointHistogramMutualInformation()

R.SetOptimizerAsGradientDescentLineSearch(5.0,
                                          2,
                                          convergenceMinimumValue=1e-4,
                                          convergenceWindowSize=5)

R.SetInterpolator(sitk.sitkNearestNeighbor)

R.SetInitialTransformAsBSpline(initial_transform,
                               inPlace=True,
                               scaleFactors=[1,2,5])
R.SetShrinkFactorsPerLevel([4,2,1])
R.SetSmoothingSigmasPerLevel([4,2,1])

R.AddCommand( sitk.sitkIterationEvent, lambda: command_iteration(R, initial_transform) )
R.AddCommand( sitk.sitkMultiResolutionIterationEvent, lambda: command_multi_iteration(R) )

final_transform = R.Execute(fixed_image, moving_image)

print("-------")
print(initial_transform)
print(final_transform)
print("Optimizer stop condition: {0}".format(R.GetOptimizerStopConditionDescription()))
print(" Iteration: {0}".format(R.GetOptimizerIteration()))
print(" Metric value: {0}".format(R.GetMetricValue()))

Initial Number of Parameters: 3993
--------- Resolution Changing ---------
itk::simple::BSplineTransform
 BSplineTransform (0x5a44a2dedae0)
   RTTI typeinfo:   itk::BSplineTransform<double, 3u, 3u>
   Reference Count: 9
   Modified Time: 27722
   Debug: Off
   Object Name: 
   Observers: 
     none
   CoefficientImage: [ 0x5a44a58f7450, 0x5a44a58f5450, 0x5a44a54315c0 ]
   TransformDomainOrigin: [1.5, 1.5, 1.5]
   TransformDomainPhysicalDimensions: [127, 127, 127]
   TransformDomainDirection: 1 0 0
0 1 0
0 0 1

   TransformDomainMeshSize: [8, 8, 8]
   GridSize: [11, 11, 11]
   GridOrigin: [-14.375, -14.375, -14.375]
   GridSpacing: [15.875, 15.875, 15.875]
   GridDirection: 1 0 0
0 1 0
0 0 1


  0 =   -0.18378
  1 =   -0.18376
Optimizer stop condition: GradientDescentLineSearchOptimizerv4Template: Maximum number of iterations (2) exceeded.
 Iteration: 2
 Metric value: -0.18377610716608403
--------- Resolution Changing ---------
itk::simple::BSplineTransform
 BSplineTransform (0x5a44a2de

In [ ]:
resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed_image);
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(100)
resampler.SetTransform(final_transform)

transformed_moving = resampler.Execute(moving_image)

'simg1 = sitk.Cast(sitk.RescaleIntensity(fixed_image), sitk.sitkUInt8)\nsimg2 = sitk.Cast(sitk.RescaleIntensity(transformed_moving), sitk.sitkUInt8)\ncimg = sitk.Compose(simg1, simg2, simg1//2.+simg2//2.)\nsitk.Show( cimg, "Image Registration Composition" )'

In [ ]:
# convert back to point cloud
def sitk_image_to_point_cloud(sitk_image):
    image_array = sitk.GetArrayFromImage(sitk_image)
    indices = np.argwhere(image_array > 0)

    spacing = sitk_image.GetSpacing()
    origin = sitk_image.GetOrigin()
    direction = np.array(sitk_image.GetDirection()).reshape(3, 3)

    # Transform indices to physical coordinates
    points = np.dot(indices * spacing, direction.T) + origin
    return points


transformed_moving_cloud = sitk_image_to_point_cloud(transformed_moving)

fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=transformed_moving_cloud[:, 0],
    y=transformed_moving_cloud[:, 1],
    z=transformed_moving_cloud[:, 2],
    mode='markers',
    marker=dict(size=3, color='red', opacity=0.8),
    name="Cluster 1 transformed to subject 0"
))

# Add the second cluster
fig.add_trace(go.Scatter3d(
    x=landmarks_0[:, 0],
    y=landmarks_0[:, 1],
    z=landmarks_0[:, 2],
    mode='markers',
    marker=dict(size=3, color='blue', opacity=0.8),
    name="Cluster suject 0"
))

fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'),
    title="3D Point Cloud Visualization"
)

fig.show()